In [1]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path

from src.data import Saver
from src.plotting import analysis
from src.utils import physics

## Loading results

In [2]:
dir_path = Path('results/')
name = 'test_tube.h5'

In [3]:
store = pd.HDFStore(dir_path/name)

In [4]:
# defining each variable
collisions_per_cell = store['collisions_per_cell'] 
df = store['df']
pmax_per_cell = store['pmax_per_cell']
averages_per_cell = store['averages_per_cell']
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']
collisions_with_walls = store['collisions_with_walls']

In [5]:
# particle mass 
mass = physics.get_mass_part(53, 53, 74) # mass of the particle
# time step
dt = 1e-6 
# ratio between real particles and simulated macro-particles
mr = 3.200000e+07

verbose=False

In [6]:
gamma = 5/3.
molecular_mass = 0.1269 # kg/mol
temperature = 300 # K - the supposed injection temperature
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for iodine [I] at temperature 300 K : {sound_vel} m/s')

Speed of sound for iodine [I] at temperature 300 K : 180.99192585903475 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing frames to plot

In [7]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.8*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Particles analysis
### Plotting number of particles evolution

In [8]:
fig, ax = plt.subplots()
analysis.nb_particles_evolution(ax, df, times = dt*unique_index);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean number of particles per cell

In [9]:
averages = averages_per_cell.groupby(averages_per_cell.index).sum() # returns a pandas Series

In [10]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(averages.index*dt, averages);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Final state

In [11]:
fig, ax = plt.subplots()
analysis.state(ax, df.loc[df.index == frames[-1]], c = None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [12]:
fig, axes = analysis.velocity_distribution(df, frames, bins = 50, density = True, sharex = False, sharey = False);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Temperature evolution

First - not by cell but for the whole system.

In [13]:
# this takes LOTS of time
df['v'] = df.apply(analysis.speed_norm, axis = 1)
speed = df['v']

In [14]:
speed.head()

0    252.009079
0    352.098271
0    314.778477
0    362.746072
0    216.704739
Name: v, dtype: float64

In [17]:
v2 = np.array([scipy.stats.moment(speed.loc[speed.index == k].values, moment = 2) for k in frames])

In [18]:
temperatures = v2  * mass/(3*physics.BOLTZMAN_CONSTANT)

In [19]:
fig, ax = plt.subplots()
ax.plot(frames*dt,temperatures)
analysis.set_axis(ax, x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
sound_speed = np.array([physics.speed_of_sound(molecular_mass, temp, gamma) for temp in temperatures])

In [26]:
out_vel = df['vx']
out_vel = out_vel.iloc[np.isin(out_vel.index, frames)]
out_vel = out_vel.groupby(out_vel.index).mean()

In [31]:
fig, ax = plt.subplots()
ax.plot(frames*dt,sound_speed)
analysis.set_axis(ax, x = 'time', y = 'sound speed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig, ax = plt.subplots()
ax.plot(frames*dt,out_vel)
analysis.set_axis(ax, x = 'time', y = 'vx')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity axis distribution per *cell*

In [97]:
# spatial_hist2d plots an hist per cell defined by the params below
# it is a 2D plot of size y_res * x_res
# where each plot (i,j) takes the particle between i*x_step < x < (i+1)*x_step and j*y_step < y < (j+1)*y_step
# for i varying between between 0 and x_res-1, and j varying between between 0 and y_res-1
# You should not use too many cells because it quickly becomes very hard to view.
# Params
x_res = 10 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001 

# Plotting - vx
fig, axes = analysis.spatial_hist2d(df, frames, val = 'vx', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Plotting - vy
fig, axes = analysis.spatial_hist2d(df, frames, val = 'vy', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Collision with walls - analysis

In [90]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions_with_walls.index*dt, collisions_with_walls*mr);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## DSMC collisions analysis

### Particles collisions analysis

In [33]:
# start by summing over all cells
collisions = collisions_per_cell.groupby(collisions_per_cell.index).sum() # returns a pandas Series
print('Total number of collision in the simulation (in reality) : {} ({:e})'.format(np.sum(collisions),mr*np.sum(collisions)));

Total number of collision in the simulation (in reality) : 919.0 (2.940800e+10)


In [89]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions.index*dt, collisions*mr);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean acceptance probability

In [17]:
proba = total_proba/collisions
print('Mean proba : {:e}'.format(np.mean(proba)))
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(proba.index*dt, proba, label = 'proba');

Mean proba : 4.073878e-01


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Maximum probability

In [18]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum() # returns a pandas Series

In [19]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(pmax.index*dt, pmax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Mean distance between two colliding particles

In [20]:
fig, ax = plt.subplots()
dist = total_distance/collisions
print('Mean distance : {:e} m'.format(np.mean(dist)))
analysis.set_axis(ax, x = 'time', y = 'distance')
ax.plot(dist.index*dt, dist);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean distance : 5.211814e-04 m


## Mean free path 
The formula for the mean free path is : $\lambda = \frac{v_{mean}}{\nu}$, where : 
* $v_{mean}$ is the mean velocity ;
* $\nu$ is the frequency of collision for ONE particle.

We have : $\frac{1}{\nu} = \frac{N_c dt}{2 N}$ with $N_c$ the number of particle in the cell, and $N$ the number of collisions.

This yields :  $\lambda = \frac{v_{mean}N_c dt}{2N}$. There is no need for $m_r$ here as both $N_c$ and $N$ would require the multiplication to have the current number in the system.

We start by doing it for the whole system.

In [34]:
x_res = 10 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001

In [35]:
v_mean = df['v'].groupby(df.index).mean()

In [36]:
v_mean = v_mean.loc[np.isin(v_mean.index,collisions.index)]
nb_particles_in_system = df['x'].groupby(df.index).agg('count').loc[np.isin(unique_index,collisions.index)]

In [37]:
mfp = 0.5*nb_particles_in_system*dt*v_mean/(collisions)

In [38]:
fig, ax = plt.subplots()
ax.plot(collisions.index*dt, mfp)
analysis.set_axis(ax, x = 'time', y = 'mean free path')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particle analysis

In [39]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Bonus - Not necessarily recommended and takes lots of time
Next is the plot of the system for each saved step. 

In [23]:
# creating speed_norm (for the color)
df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df):
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['speed_norm'])
    
    ax.set_title('{}/{}'.format(num+1, iterations), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['speed_norm'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)

ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
ax.axis('equal')

interval = 40 # 25 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=iterations, fargs=(df, ), save_count=iterations)
anim.save(dir_path/'system_state_evo.mp4', dpi = 300);